In [1]:
import pyliger

/Users/rubbit/anaconda3/envs/genetics_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
/Users/rubbit/anaconda3/envs/genetics_env/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
/Users/rubbit/anaconda3/envs/genetics_env/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
/Users/rubbit/anaconda3/envs/genetics_env/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previou

In [2]:
print(pyliger.__file__)

/Users/rubbit/Documents/GitHub/pyliger/src/pyliger/__init__.py


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import pyliger as pl
from scipy.stats import pearsonr, spearmanr
from scipy.optimize import linear_sum_assignment
import os
import anndata as ad

In [3]:
X_dataset1 = pd.read_csv("/Users/rubbit/Desktop/Brown/PHP 2691/Project/X_dataset1.csv").values
X_dataset2 = pd.read_csv("/Users/rubbit/Desktop/Brown/PHP 2691/Project/X_dataset2.csv").values
coords1 = pd.read_csv("/Users/rubbit/Desktop/Brown/PHP 2691/Project/coords_dataset1.csv").values
coords2 = pd.read_csv("/Users/rubbit/Desktop/Brown/PHP 2691/Project/coords_dataset2.csv").values

In [4]:
# Create AnnData object
adata1 = ad.AnnData(X_dataset1)
adata1.obs_names = [f"Spot{i}" for i in range(X_dataset1.shape[0])]
adata1.var_names = [f"Gene{j}" for j in range(X_dataset1.shape[1])]

# ✅ Required by pyLiger:
adata1.obs.index.name = "cell"
adata1.var.index.name = "gene"
# ✅ Required by PyLiger
adata1.uns["sample_name"] = "simulated1"

# Create AnnData object
adata2 = ad.AnnData(X_dataset2)
adata2.obs_names = [f"Spot{i}" for i in range(X_dataset2.shape[0])]
adata2.var_names = [f"Gene{j}" for j in range(X_dataset2.shape[1])]

# ✅ Required by pyLiger:
adata2.obs.index.name = "cell"
adata2.var.index.name = "gene"
# ✅ Required by PyLiger
adata2.uns["sample_name"] = "simulated2"


In [5]:
from sklearn.neighbors import NearestNeighbors

In [6]:
def build_spatial_laplacian(coords, n_neighbors=8, sigma=1.0):
    """Build unnormalized graph Laplacian from spatial coords."""
    nbrs = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(coords)
    dists, idxs = nbrs.kneighbors(coords)
    n = coords.shape[0]
    W = np.zeros((n, n), dtype=float)
    for i in range(n):
        for dist, j in zip(dists[i, 1:], idxs[i, 1:]):
            w = np.exp(-dist**2 / (2 * sigma**2))
            W[i, j] = W[j, i] = w
    D = np.diag(W.sum(axis=1))
    return D - W
coords_list = [coords1, coords2]
L_list = []
for i in range(2):
    coords = coords_list[i]
    L_list.append(build_spatial_laplacian(coords, n_neighbors=8, sigma=2))

In [15]:
# 3. Create PyLiger object (as a dictionary of dataset name → AnnData)
liger_int = pyliger.create_liger([adata1, adata2], remove_missing=False)
pyliger.normalize(liger_int, remove_missing=False)
pyliger.select_genes(liger_int, var_thresh=0)
#liger.adata_list
pyliger.scale_not_center(liger_int, remove_missing=False)
pyliger.optimize_ALS(liger_int, L_list=L_list, spatial_gamma=80, bi_gamma=0.5, nrep = 10, k = 4)

ALS rep 10: 100%|██████████| 30/30 [00:01<00:00, 26.70it/s]


In [21]:
liger_int.adata_list

[AnnData object with n_obs × n_vars = 625 × 1395
     obs: 'nUMI', 'nGene', 'dataset'
     var: 'gene_sum', 'gene_sum_sq', 'nCell', 'norm_sum', 'norm_sum_sq', 'norm_mean', 'norm_var'
     uns: 'sample_name', 'var_gene_idx'
     obsm: 'H'
     varm: 'W', 'V'
     layers: 'norm_data', 'scale_data',
 AnnData object with n_obs × n_vars = 625 × 1395
     obs: 'nUMI', 'nGene', 'dataset'
     var: 'gene_sum', 'gene_sum_sq', 'nCell', 'norm_sum', 'norm_sum_sq', 'norm_mean', 'norm_var'
     uns: 'sample_name', 'var_gene_idx'
     obsm: 'H'
     varm: 'W', 'V'
     layers: 'norm_data', 'scale_data']

In [17]:
H_dataset1 = liger_int.H[0]
H_dataset2 = liger_int.H[1]
H1_df = pd.DataFrame(
    H_dataset1
)
H1_df.to_csv("/Users/rubbit/Desktop/Brown/PHP 2691/Project/H_liger_spatial_1.csv")

H2_df = pd.DataFrame(
    H_dataset2
)
H2_df.to_csv("/Users/rubbit/Desktop/Brown/PHP 2691/Project/H_liger_spatial_2.csv")

In [18]:
W_dataset1 = liger_int.W[0]

In [ ]:
help()

In [19]:
W1_df = pd.DataFrame(
    W_dataset1
)
W1_df.to_csv("/Users/rubbit/Desktop/Brown/PHP 2691/Project/W_liger_spatial_1.csv")

In [22]:
V_dataset1 = liger_int.V[0]
V1_df = pd.DataFrame(
    V_dataset1
)
V1_df.to_csv("/Users/rubbit/Desktop/Brown/PHP 2691/Project/V_liger_spatial_1.csv")